BetaBuddy
=====

An Automatic Beta Cell Analysis Pipeline 
---
<hr>

#### Written with equal contibution by: Anne Alsup *anne.alsup@mavs.uta.edu* & Kelli Fowlds *kelli.fowlds@mavs.uta.edu*.

Major compontents of the Cellpose script was written by Pradeep Rajasekhar. The [Cellpose GitHub](https://github.com/MouseLand/cellpose) along with the original [GoogleColab](https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/Cellpose_cell_segmentation_2D_prediction_only.ipynb) by Pradeep are linked. 

Please read through all instructions given in each step cell. This a customizable pipeline, but the given format must be followed!

If you run into any issues, please let us know in our [BetaBuddy GitHub](https://github.com/jacobluber/BetaBuddy)!


## Step 1. nd2 to TIFF conversion
<hr>

1. The CellPoseImg directory will be emptied before each run. **Save or move any images you want to keep before running this step.**

2. If you already have TIFF, JPG, or PNG files, you will have to: 
    + Comment out (add a *#* before the line) the two lines that begin with *!./bftools/bfconvert* in **Cell1**.

3. **If you aren't using a DAPI channel, comment out the first *!./bftools* line.** 

4. Change the arguements for bftools
    +  Change the *./1V_Ca_Spike_Post.nd2* and/or *./DAPI_Post.nd2* to your ND2 file name(s). Keep the **./** before your file name as this lets the program know you file is located in the current directory.
    + Change the *1VSpiking.tif* and/or *1VDAPI.tif* name(s) to what you want your TIFF to be named. This arguement does *not* need the **./**.

*Note: Your filenames cannot have any spaces when using a LINUX OS. Replacing spaces with '_' is common practice* <br>
*Note 2: Your image must be in a stack format*

In [ ]:
# Cell1
# Follow this format: !./bftools/bfconvert -overwrite ./FILENAME.nd2 NEWFILENAME.tif

!./bftools/bfconvert -overwrite ./DAPI_Post.nd2 1VDAPIPost.tif #comment out for no DAPI channel
!./bftools/bfconvert -overwrite ./1V_Ca_Spike_Post.nd2 1V_Ca_Spike_Post.tif

import os
import shutil

# Emptying the CellPoseImg directory that will be the output directory for the merged image
imgsave_dir = "./CellPoseImg"
if not os.path.exists(imgsave_dir):
    print("No CellPoseImg directory found. Creating a new one.")
    os.mkdir(imgsave_dir)
else:
  print("Existing CellPoseImg directory found. Deleting it.")
  shutil.rmtree(imgsave_dir)
  os.mkdir(imgsave_dir)

##  Step 2. Merge single DAPI stain with fluorescent image stack</h2>
<hr>

* The input arguement for this step will follow this format **'/my/path/end/with/slash/ 1V_Spike.tiff DAPI.tiff DAPI'**
    + Every arguement is seperated with a space. Follow the steps below for all four arguements.
1. Your current directory path ending with a backslash (/my/path/end/with/slash/).
2. Your fluorescent image stack name (1V_Spike.tiff).
3. Your DAPI image name (DAPI.tiff).
    + *Note if you do not have a DAPI stack leave the argument as DAPI.tiff or any string. The arguement requires 4 inputs*
4. This determines if you have a DAPI image stack. Change this input argument to **DAPI** if you have a sperate DAPI stack needing to be merged. If you do not have a DAPI stack, change this to **NoDAPI**.

**Note: If you don't have a DAPI channel, move the *Scaled_Beta_Cells* TIFF file to the CellPoseImg directory *AFTER* you run this step.**


In [ ]:
# Cell2

# Change arguments below
!./Fiji.app/ImageJ-linux64 --headless -macro ./DAPIMerge.ijm '/home/BetaBuddy/ 1V_Ca_Spike_Post.tif 1VDAPIPost.tif DAPI'

Step 3. Automatic cell segmentation with Cellpose 
----
<hr>

In [ ]:
# Cell3
# Configuring Cellpose, adding dependencies, and checking GPU access
# https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/Cellpose_cell_segmentation_2D_prediction_only.ipynb
import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import ipywidgets as widgets
import numpy as np
from IPython.display import display
import shutil
import re

from cellpose import models,core

if core.use_gpu()==False: 
  #Warnings from the ZeroCost StarDist notebook
  print("You do not have GPU access. Cellpose will run through CPU.")
  use_GPU=False
else:
  print("You have access to the GPU.")
  use_GPU=True

!nvidia-smi
print("All dependencies loaded")

### Step 3.1: Set parameters for image format, file type, and input directory 

<hr>

#### Widgets will appear in the output after running the cell. It's helpful to run the widget cells using *ctrl+Enter*, so Jupyter doesn't skip to the next cell.

#### Note: The button will become dark gray when selected.

1. Choose how your images are formatted. *Note: Check your CellPoseImg directory if you are unsure what your image format currently is.*
    + Click on 'Frames Seperated' if all your frames are individual files. 
    + Click on 'Image Stack' if all your frames are in one file.
2. Choose your image format. Different formats are supported, but TIFF is recommended.
3. Confirm the input directory is correct. 

In [ ]:
# Cell4
# Creating input widgets
image_form = widgets.ToggleButtons(
    options=['Frames Seperated', 'Image Stack'],
    description='Select One', 
)
display(image_form)

file_type = widgets.ToggleButtons(
    options=['TIFF', 'JPEG', 'PNG'],
    description='Select One', 
)
display(file_type)


input_dir = os.getcwd() +'/CellPoseImg/'
print("Input directory: " + input_dir)

In [ ]:
# Cell5
print("Image form: "+image_form.value)
print("Image file type: "+ file_type.value)

imgsep = image_form.value

if file_type.value == "TIFF":
    img_format = ".tif"
elif file_type.value =="JPG":
    img_format = ".jpg"
else:
    img_format = ".png"

fname1 = os.listdir(input_dir)
fname = [item for item in fname1 if item.endswith(img_format)]    
    
# Image stack requires all frames to be seperated into individual images
if imgsep =="Image Stack":
    imgstack = skimage.io.imread(input_dir+fname[0])
    dims = imgstack.shape
    print(dims)
    fname = re.sub(rf"\b{img_format}\b","",fname[0])
    
    # Create or empty merged sequence dir
    imgsave_dir = input_dir+"MergedSeq/"
    if not os.path.exists(imgsave_dir):
        os.mkdir(imgsave_dir)
    else:
        print("Existing Merged Sequence Directory found. Deleting it.")
        shutil.rmtree(imgsave_dir)
        os.mkdir(imgsave_dir)
    
    # Creating Image Sequence of TIFF Stack
    count = 0
    for i in range(dims[0]):
        skimage.io.imsave(imgsave_dir+fname+str(count)+".tif",imgstack[i], check_contrast=False)
        print(count)
        count += 1  
    fname1 = os.listdir(imgsave_dir)
    fname = [item for item in fname1 if item.endswith(img_format)] 
    
else: 
    imgsave_dir = input_dir

# Creating/Emptyingc Mask folder
save_dir = input_dir+"Masks/"
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
else:
    print("Existing Mask Directory found. Deleting it.")
    shutil.rmtree(save_dir)

    
if(len(fname)==0):
    print("Number of images loaded: %d." %(len(fname)))
    print("Cannot read image files. Check if folder has images")
else:
    print("Number of images loaded: %d." %(len(fname)))
    
# Reading all images and displaying one at random 
imgs = []
for im in range(len(fname)):
    im = skimage.io.imread(imgsave_dir + fname[im])
    n_dim=len(im.shape) #shape of image
    dim=im.shape #dimensions of image
    channel=min(dim) #channel will be dimension with min value usually
    channel_position=dim.index(channel)
    # If no of dim is 3 and channel is first index, swap channel to last index
    if n_dim==3 and channel_position==0: 
        im=im.transpose(1,2,0)
        dim=im.shape
    imgs.append(im)
print("Number of images read: " + str(len(imgs)))
print("Example Image:")
random_idx = random.choice(range(len(imgs)))
x=imgs[random_idx]
n_dim=len(x.shape)
file_name=os.path.basename(fname[random_idx])
print(file_name+" has "+str(n_dim)+" dimensions/s")
if n_dim==3:
    channel_image=x.shape[2]
    fig, axs = plt.subplots(1, channel_image,figsize=(12,5))
    print("Image: %s" %(file_name))
    for channel in range(channel_image):
        axs[channel].imshow(x[:,:,channel])
        axs[channel].set_title('Channel '+str(channel+1),size=5)
        axs[channel].axis('off')
    fig.tight_layout()
elif n_dim==2:
    print("One Channel")
    plt.imshow(x)
else:
    print("Channel number invalid or dimensions wrong. Image shape is: "+str(x.shape))

### Step 3.2: Set Cellpose parameters and run test segmentation 
<hr>


1. Choose a model option 
    + Cytoplasm and Cytoplasm2 will work for whole cell imaging. 
    + Nuclei will work well for images with only DAPI staining. 
    + *Note: Only nuclei segmentation may not perform well for subsequent tracking and analysis steps.*

2. Choose the channel that contains the entire cell to segment. You can reference the images above to find the correct channel.
    + *Keep the channel at 0 if you only have one channel.*

3. Choose the channel with your DAPI stain. **If you do not have a DAPI stain, keep the channel at 0.**
4. Set Cell Probability Threshold  
    + Decrease this threshold if your masks are too small or if Cellpose is not detecting enough cells.
    + Increase this threshold if Cellpose is detecting too many cells. 

5. Set Flow Threshold 
    + Decrease this parameter if Cellpose is not detecting enough cells. 
    + Increase this parameter if Cellpose is detecting too many cells.      

In [ ]:
# Cell6
Model_Choice = widgets.ToggleButtons(
    options=['Cytoplasm', 'Cytoplasm2', 'Nuclei'],
    value = 'Cytoplasm2',
    description='Select One', 
)
display(Model_Choice)

segchan=widgets.Dropdown(
    options=['0','1', '2', '3'],
    value='2',
    description='Cell Channel:',
    disabled=False,
)
display(segchan)

qDapi=widgets.Dropdown(
    options=['0','1', '2', '3'],
    value='3',
    description='DAPI Channel:',
    disabled=False,
)
display(qDapi)

cellprob=widgets.FloatSlider(
    value=-3,
    min=-6, 
    max=6, 
    step=1.0, 
    description='Cell Probability'
)
display(cellprob)

flowthresh = widgets.FloatSlider(
    value=0.9,
    min=0.1,
    max=1.1, 
    step=0.1, 
    description='Flow Threshold'
)
display(flowthresh)

In [ ]:
# Cell7
import torch
print("Model choice is: " + Model_Choice.value)
print("Segmentation channel is: " + str(segchan.value))
if qDapi.value == 0:
    Use_nuclear_channel = False
    print("No DAPI Channel")
else:
    Use_nuclear_channel = True
    print("DAPI channel is: " + qDapi.value)

print("Cell probability threshold: " + str(cellprob.value))
print("Flow threshold: " + str(flowthresh.value))
      
model_choice = Model_Choice.value
segment_channel = int(segchan.value)
nuclear_channel = int(qDapi.value)

if model_choice=="Cytoplasm":
  model_type="cyto"
elif model_choice=="Cytoplasm2":
  model_type="cyto2"
elif model_choice=="Nuclei":
  model_type="nuclei" 

# channels = [cytoplasm, nucleus]
if model_choice not in "Nucleus":
  if Use_nuclear_channel:
    channels=[segment_channel,nuclear_channel]
  else:
    channels=[segment_channel,0]
else: #nucleus
  channels=[segment_channel,0]

# DEFINE CELLPOSE MODEL
model = models.Cellpose(gpu=False, model_type=model_type)
print(model)
diameter = None
from skimage.util import img_as_ubyte

flow_threshold=flowthresh.value
cellprob_threshold=cellprob.value

# You can change the test image here
img1=imgs[1]
import cv2
masks, flows, styles, diams = model.eval(img1,diameter=None,flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)

# DISPLAY RESULTS
from cellpose import plot
maski = masks
flowi = flows[0]

#convert to 8-bit if not so it can display properly in the graph
if img1.dtype!='uint8':
  img1=img_as_ubyte(img1)

fig = plt.figure(figsize=(24,8))
plot.show_segmentation(fig, img1, maski, flowi, channels=channels)
plt.tight_layout()
plt.show()

In [ ]:
# Cell8
# Save mask images in Masks directory under CellPoseImgs
print("Save Directory is: ",save_dir)
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)

for img_idx in range(len(fname)):
    file_name=fname[img_idx]
    print("\nSegmenting: ",file_name)
    mask, flow, style, diam = model.eval(imgs[img_idx], diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)
    # Save name for masks
    mask_output_name=save_dir+"MASK_"+file_name+".tif"
    # Save mask as 16-bit in case this has to be used for detecting than 255 objects
    # A 16-bit image will look black until opened in software such as ImageJ
    # Change "uint16" to "uint8" if you want the preview to be visible 
    mask=mask.astype(np.uint16)
    skimage.io.imsave(mask_output_name, mask, check_contrast=False)
 
print("\nSegmentation complete and files saved :)")

Step 4. Cell registration and tracking 
----
<hr>

1. The first terminal command will merge the mask images with their corresponding orginal beta cell image to begin cell registration. 
    + **You will need to change the *'/home/BetaBuddy/'* to your current directory**
2. The second terminal command will push our mask/original image stack through the Fiji plugin TrackMate. A few short printouts should show up with a very long log of all the track locations and frames afterwards. 
</br> 


### Outputs

* An XML titled *MaskMerged.xml* will be saved in the current directory along with the *MaskMerged.tif*. You can load this into Fiji through *Plugins -> Tracking -> Load TrackMate File* and see the calculated tracks. You can also open the *Track* tab and click on TrackIDs to see their location within the TrackMate GUI. 
* Raw data will be saved in a CSV named "experimentjup.csv". 

*Note: The TrackIDs in the CSV files will be off by one.*


In [ ]:
# Cell9

!./Fiji.app/ImageJ-linux64 --headless -macro ./MaskMerge.ijm '/home/BetaBuddy/'
!./Fiji.app/ImageJ-linux64 --headless ./LabelLapLinuxTerminalJup.py

Step 5. Background Subtraction 
----

<hr>

* Random pixel locations will be chosen and compared to every mask frame to ensure the location is not within a cell. 
* If the location is not within a cell, the pixel value from every frame will be saved in a csv named "backgroundsub". 
* This will be used in the next step for background subtraction for signal analysis.
* **The Scaled_Beta_Cells.tiff image can only be one channel for this script to work correctly.**

In [ ]:
# Cell10

import os
from skimage import io
from matplotlib import pyplot as plt
import numpy as np
import csv
import random
from statistics import mean

#get path for image loading
dir_path = os.getcwd()

#load stacked original images and mask images
imspike = io.imread(dir_path+'/Scaled_Beta_Cells.tif')
immask = io.imread(dir_path+'/MaskStack.tif')

# Get dimensions for each image (frames, 900, 900, # of channels)
dims=imspike.shape
dimsm=immask.shape
print("Image dimensions: " + str(dims))
# How many pixels to sample
spotnum = 100

# Create array for pixel values of all 100 spots
x = []
for index in range(0,dims[0]):
    x.append([None]*spotnum)

count = 0
# Array to house locations that are not masks
goodlocs=[]
tmpx = []
while count<spotnum:
    loc = []
    imgval = []
    #random pixel location
    x1tmp = random.randrange(0,int(len(imspike[1])))
    y1tmp = random.randrange(0,int(len(imspike[1])))
    imgloc = [int(x1tmp),int(y1tmp)]
    tmpx = np.array([None]*int(dims[0]))
    for img in range(dims[0]):
        #pixel intensity from all three channels
        imgval = imspike[img,imgloc[0],imgloc[1]] #8bit
        maimgval = immask[img,imgloc[0],imgloc[1]]
        # If intensity on mask image is 0, the pixel location is *not* within a cell 
        if np.all(maimgval==0):
            tmpx[img]=imgval 
        # If intensity on mask image is *not* 0, all temporary variables are emptied   
        else:
            maimagval = []
            imgval = []
            imgloc=[]
            break
    if None in tmpx:
      # Empty out temporary array if there are any "None"s left
      tmpx = []
    else: 
      # Save temporary array into large array will all intensity values  
        for do in range(dims[0]):
            x[do][count] = tmpx[do]
        # Save pixel location  
        goodlocs.append(imgloc)
        count += 1

# Header row names for csv
heads = []
for namez in range(spotnum):
    heads.append('Spot '+ str(namez))

# Saving pixel locations for reference
spotz = []
for locz in range(spotnum):
    spotz.append('('+str(goodlocs[locz][1])+'/'+str(goodlocs[locz][0])+')')

print("Locations of background subtraction: " + str(spotz))

# Save intensity values of all 100 pixel locations across all frames
filename = dir_path+"/backgroundsub.csv"
with open(filename,'w') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter = ',',lineterminator='\r')
    csvwrite2 = csv.writer(csvfile, delimiter = ' ',lineterminator='\r')
    csvwriter.writerow(heads)
    csvwriter.writerow(spotz)
    for i in range(len(x)):
        csvwriter.writerow(x[i])

## Step 6. Statistical Analysis 
<hr>

* We can now run the statistical analysis in an R script! 
* The cell below should create three figures and a CSV file, named Experimental_Data.csv, you can use in custom analyses. All will be saved in your current directory. 
* The final CSV file has only tracks that span the full image stack and identifies tracks in the top 20% of signal variance.
</ul>


In [ ]:
# Cell11

!Rscript Beta_Buddy_Stats.R